In [54]:
%load_ext autoreload
%autoreload 2

import sys  
sys.path.insert(0, 'code')

import pandas as pd
import xlsxwriter
import os
import os.path as osp
import numpy as np

#my code
from load_utils import loadExcel
from sheet_number_of_travels import createSheetNumberOfTravels
from sheet_times import createSheetTimes
from sheet_number_of_cars import createSheetNumberOfCars, computeData, createFakeDataset
import yaml

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
with open('parameters.yaml') as f:
    params = yaml.safe_load(f)
N = params['number_of_cameras']

df_save = loadExcel('input/input.xlsx', N=params['number_of_cameras'])

In [66]:
df = df_save.copy(deep=True)

In [125]:
if osp.isfile('out.xlsx'):
    os.remove('out.xlsx')

with open('parameters.yaml') as f:
    params = yaml.safe_load(f)
    
try:
    writer = pd.ExcelWriter('out.xlsx', engine='xlsxwriter')
    createSheetNumberOfTravels(df.copy(True), writer, params)
    writer.close()
except Exception as e:
    print(e)
    writer.close()

Capture_time
License_plate Direction              
21            16                    1
455           8                     1
676           10                    1
1032          9                     1
1074          10                    1
...                               ...
unknown       14                  220
              15                  125
              16                   60
              17                   19
              18                  253

[76915 rows x 1 columns]

In [94]:
temp = df[df['Direction'] == 1]

# solve correct spz
temp = temp[temp['License_plate'] != 'unknown']
temp = temp.groupby(by='License_plate').count().groupby(by='Capture_time').count()
temp

,Direction
Capture_time,
1,2340
2,39


In [100]:
N = params['number_of_cameras']

dick = {
    'Direction': [],
    'SPZ': [],
    'Count': []
}

# add fake column to be removed later, order of operations should be kept because SPZs ordering matter
for spz in range(1, 11): #always show atleast 10 SPZs
    dick['Direction'].append('remove me')
    dick['SPZ'].append(spz)
    dick['Count'].append(0)

for direction in df['Direction'].unique():
    temp = df[df['Direction'] == direction]

    # solve unknown spz
    temp_unknown = temp[temp['License_plate'] == 'unknown']
    dick['Direction'].append(direction)
    dick['SPZ'].append(9999)
    dick['Count'].append(len(temp_unknown))

    # solve correct spz
    temp = temp[temp['License_plate'] != 'unknown']
    temp = temp.groupby(by='License_plate').count().groupby(by='Capture_time').count()
    temp = temp.reset_index()

    # column names lost their oringal meaning... 
    for spz, count in zip(temp['Capture_time'], temp['Direction']):
        dick['Direction'].append(direction)
        dick['SPZ'].append(spz)
        dick['Count'].append(count)

out = pd.DataFrame.from_dict(dick)
SPZs = out['SPZ'].unique()
SPZs.sort()
SPZs = SPZs.tolist()
SPZs[-1] = 'unknown'

out = out.pivot(index='SPZ', columns='Direction', values='Count').fillna(0)
out = out.drop(columns=['remove me'])

In [58]:
# add fake data to force dataframe layout, aka have at least one entry for every possible combination
fake_df = createFakeDataset(params['number_of_cameras'])
df = df.append(fake_df, ignore_index=True)

# separate dataset into two parts based on license plate count
plate_counts = df["License_plate"].value_counts()
plate_counts['unknown'] = 0  # fake unknown values, so they are selected into single dataset
plate_count = plate_counts[plate_counts > 1]

df_multiple = df[df['License_plate'].isin(plate_count.index)]
df_single = df[~df['License_plate'].isin(plate_count.index)]

# create another dataset moved by one and merge it. Aka create pairs of all following directions
df_multiple = df_multiple.sort_values(by=['License_plate', 'Capture_time']).reset_index(drop=True)
df_multiple['joinIndex'] = df_multiple.index
df_multiple_moved = df_multiple.copy(deep=True)
df_multiple_moved['joinIndex'] = df_multiple_moved.index - 1
df_multiple_join = df_multiple.merge(df_multiple_moved, how='inner', on=['joinIndex', 'License_plate'],
                                     suffixes=['_from', '_to'])

In [63]:
df_multiple_join[(df_multiple_join['Direction_from'] == 13) & (df_multiple_join['Direction_to'] == 18)]

,License_plate,Capture_time_from,Direction_from,joinIndex,Capture_time_to,Direction_to


In [64]:
fake_df

,Capture_time,License_plate,Direction
0,2021-10-28 00:00:00,fake_1,1
1,2021-10-28 00:00:00,fake_1_1,1
2,2021-10-28 01:00:00,fake_1_1,1
3,2021-10-28 00:00:00,fake_1_2,1
4,2021-10-28 01:00:00,fake_1_2,2
...,...,...,...
625,2021-10-28 01:00:00,fake_18_15,15
626,2021-10-28 00:00:00,fake_18_16,18
627,2021-10-28 01:00:00,fake_18_16,16
628,2021-10-28 00:00:00,fake_18_17,18
